## Data Input

**Current Data Availability:**

- Monthly excess returns for **6 anomalies**.
- **Fama–French factors** (MKT, SMB, HML). _(No changes required, use as is)_.
- **Regime Classification:**
  - **Pre-Crisis:** 2003-01 to 2007-11
  - **Crisis:** 2007-12 to 2009-06
  - **Post-Crisis:** 2009-07 to 2014-05

---

## Time-Series Regression Analysis

This model evaluates whether the risk-adjusted returns of each anomaly changed significantly during the crisis regime.

### Model Specifications

**Model 1: Pre-crisis vs. Crisis**
$$R_t = \alpha + \gamma_1 \cdot \text{CrisisDummy}_t + \beta \cdot \text{Factors}_t + \epsilon_t$$

**Model 2: Crisis vs. Post-crisis**
$$R_t = \alpha + \gamma_2 \cdot \text{PostCrisisDummy}_t + \beta \cdot \text{Factors}_t + \epsilon_t$$

### Variable Definitions

- **$R_t$**: The excess return of the long–short anomaly portfolio in the month.
- **$\text{CrisisDummy}_t$**: A dummy variable equal to **1** during the crisis and **0** during the pre-crisis (Model 1).
- **$\text{PostCrisisDummy}_t$**: A dummy variable equal to **1** during the post-crisis and **0** during the crisis (Model 2).
- **$\text{Factors}_t$**: Represents the standard Fama–French three-factor model: Market (MKT), Size (SMB), and Value (HML).
- **$\gamma_1, \gamma_2$**: Coefficients capturing the change in risk-adjusted anomaly return during the crisis.
- **$\epsilon_t$**: The error term.


In [1]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [2]:
excess_returns = pd.read_excel('./Regression Data/excess_returns.xlsx', index_col=0)
display(round(excess_returns,3))
excess_returns.info()

,Accruals,Asset Growth,BM,Gross Profit,Momentum,Leverage Ret,Regime
date,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis
...,...,...,...,...,...,...,...
2014-01-31,-0.014,0.003,-0.024,-0.144,-0.043,-0.010,Post-Crisis
2014-02-28,-0.008,-0.010,-0.018,0.004,0.010,0.017,Post-Crisis
2014-03-31,0.011,0.040,0.001,0.051,-0.001,0.000,Post-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Accruals      137 non-null    float64
 1   Asset Growth  137 non-null    float64
 2   BM            137 non-null    float64
 3   Gross Profit  137 non-null    float64
 4   Momentum      137 non-null    float64
 5   Leverage Ret  137 non-null    float64
 6   Regime        137 non-null    object 
dtypes: float64(6), object(1)
memory usage: 8.6+ KB


In [3]:
ff_factors = pd.read_excel('./Regression Data/fama_french_factors.xlsx', index_col='date')
display(round(ff_factors.head(),3))
print(ff_factors.info())

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Regime
date,,,,,,
2003-01-31,0,-0.027,0.019,0.008,0.001,Pre-Crisis
2003-02-28,1,-0.018,0.005,0.017,0.001,Pre-Crisis
2003-03-31,2,-0.004,0.005,-0.011,0.001,Pre-Crisis
2003-04-30,3,0.087,-0.000,-0.008,0.001,Pre-Crisis
2003-05-30,4,0.066,0.032,0.005,0.001,Pre-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  137 non-null    int64  
 1   Mkt-RF      137 non-null    float64
 2   SMB         137 non-null    float64
 3   HML         137 non-null    float64
 4   RF          137 non-null    float64
 5   Regime      137 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.5+ KB
None


In [4]:
# Merge Data
# We join the anomaly returns with the Fama-French factors.
# We only need Mkt-RF, SMB, and HML from the factors df.
df_combined = pd.merge(
    excess_returns, 
    ff_factors[['Mkt-RF', 'SMB', 'HML']], 
    left_index=True, 
    right_index=True, 
    how='inner'
)
display(round(df_combined.head(),3))

,Accruals,Asset Growth,BM,Gross Profit,Momentum,Leverage Ret,Regime,Mkt-RF,SMB,HML
date,,,,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis,-0.027,0.019,0.008
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis,-0.018,0.005,0.017
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis,-0.004,0.005,-0.011
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis,0.087,-0.000,-0.008
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis,0.066,0.032,0.005


In [5]:
# List of anomaly columns (Dependent Variables)
anomalies = ['Accruals', 'Asset Growth', 'BM', 'Gross Profit', 'Momentum', 'Leverage Ret']

In [6]:
# ---  Define the Regression Function ---

def run_regressions(data, anomalies, dummy_col_name):
    results_list = []
    
    # Independent Variables: Constant + 3 Factors + Regime Dummy
    X_cols = ['Mkt-RF', 'SMB', 'HML', dummy_col_name]
    X = sm.add_constant(data[X_cols])
    
    for anomaly in anomalies:
        # Dependent Variable
        y = data[anomaly]
        
        # Fit OLS
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
        # Store results
        results_list.append({
            'Anomaly': anomaly,
            'Gamma_Coef': model.params[dummy_col_name],
            'P_Value': model.pvalues[dummy_col_name],
            'Significant (5%)': model.pvalues[dummy_col_name] < 0.05,
            't_stat': model.tvalues[dummy_col_name],
            'Beta MKT': model.params['Mkt-RF'],
            'Beta SMB': model.params['SMB'],
            'Beta HML': model.params['HML'],
            'Alpha': model.params['const'],
            'Adjusted R2': model.rsquared_adj,
        })
        
    return pd.DataFrame(results_list)


In [7]:

# --- Run Model 1: Pre-Crisis vs. Crisis ---
# Filter: Keep only Pre-Crisis and Crisis rows
df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()
print(df_m1['Regime'].unique())

# Dummy: 1 if Crisis, 0 if Pre-Crisis
df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

results_m1 = run_regressions(df_m1, anomalies, 'CrisisDummy')
results_m1.to_excel('./Regression Results/model_1_results.xlsx', index=False)
display(round(results_m1,3))



['Pre-Crisis' 'Crisis']


,Anomaly,Gamma_Coef,P_Value,Significant (5%),t_stat,Beta MKT,Beta SMB,Beta HML,Alpha,Adjusted R2
0,Accruals,-0.013,0.138,False,-1.482,0.005,0.356,-1.068,0.005,0.245
1,Asset Growth,-0.006,0.536,False,-0.619,-0.068,0.362,0.377,-0.001,0.013
2,BM,0.018,0.243,False,1.166,0.455,0.517,0.667,0.003,0.376
3,Gross Profit,0.005,0.551,False,0.597,-0.383,-0.578,0.700,-0.002,0.257
4,Momentum,-0.060,0.160,False,-1.406,-0.687,-0.444,-0.378,0.011,0.154
5,Leverage Ret,0.008,0.186,False,1.323,-0.059,-0.232,0.439,-0.002,0.156


In [8]:
# --- Run Model 2: Crisis vs. Post-Crisis ---
# Filter: Keep only Crisis and Post-Crisis rows
df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()
print(df_m2['Regime'].unique())
# Dummy: 1 if Post-Crisis, 0 if Crisis
df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

print("\nRunning Model 2 (Crisis vs. Post-Crisis)...")
results_m2 = run_regressions(df_m2, anomalies, 'PostCrisisDummy')
results_m2.to_excel('./Regression Results/model_2_results.xlsx', index=False)
display(round(results_m2,3))

['Crisis' 'Post-Crisis']

Running Model 2 (Crisis vs. Post-Crisis)...


,Anomaly,Gamma_Coef,P_Value,Significant (5%),t_stat,Beta MKT,Beta SMB,Beta HML,Alpha,Adjusted R2
0,Accruals,0.006,0.485,False,0.699,0.101,0.230,-0.555,-0.005,0.119
1,Asset Growth,0.006,0.496,False,0.680,-0.020,0.072,0.811,-0.004,0.152
2,BM,-0.019,0.095,False,-1.671,0.201,0.618,1.081,0.016,0.335
3,Gross Profit,-0.004,0.617,False,-0.500,-0.201,-1.074,-0.135,0.006,0.197
4,Momentum,0.053,0.199,False,1.283,-0.318,-1.699,-1.359,-0.040,0.219
5,Leverage Ret,-0.006,0.343,False,-0.948,-0.059,-0.345,0.346,0.006,0.152
